In [2]:
import os

import torch
from hydra.utils import instantiate
from monai.apps import download_url
from neuro_utils.visualize import plot_scans
from torch import nn

from diffusion_3d.chestct.autoencoder.adaptive_vae.maisi.config import get_config
from diffusion_3d.datasets.ct_rate import CTRATEDataModule

In [3]:
trained_autoencoder_path = r"/raid3/arjun/checkpoints/maisi/autoencoder_epoch273.pt"
trained_autoencoder_path_url = "https://developer.download.nvidia.com/assets/Clara/monai/tutorials/model_zoo/model_maisi_autoencoder_epoch273_alternative.pt"
if not os.path.exists(trained_autoencoder_path):
    download_url(url=trained_autoencoder_path_url, filepath=trained_autoencoder_path)

In [4]:
state_dict = torch.load(trained_autoencoder_path, map_location='cpu')

/tmp/ipykernel_1222622/2833122154.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(trained_autoencoder_path, map_location='cpu')


In [5]:
model_config = {
    "_target_": "monai.apps.generation.maisi.networks.autoencoderkl_maisi.AutoencoderKlMaisi",
    "spatial_dims": 3,
    "in_channels": 1,
    "out_channels": 1,
    "latent_channels": 4,
    "num_channels": [64, 128, 256],
    "num_res_blocks": [2, 2, 2],
    "norm_num_groups": 32,
    "norm_eps": 1e-06,
    "attention_levels": [False, False, False],
    "with_encoder_nonlocal_attn": False,
    "with_decoder_nonlocal_attn": False,
    "use_checkpointing": False,
    "use_convtranspose": False,
    "norm_float16": False,
    "num_splits": 1,
    "dim_split": 2,
    'save_mem': False,
}

In [6]:
device = torch.device('cuda:0')
# Don't use cpu, server crashes because of mem and compute requirements

In [7]:
model: nn.Module = instantiate(model_config)
model.load_state_dict(state_dict)
model.to(device)
model.eval()


AutoencoderKlMaisi(
  (encoder): MaisiEncoder(
    (blocks): ModuleList(
      (0): MaisiConvolution(
        (conv): Convolution(
          (conv): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        )
      )
      (1-2): 2 x MaisiResBlock(
        (norm1): MaisiGroupNorm3D(32, 64, eps=1e-06, affine=True)
        (conv1): MaisiConvolution(
          (conv): Convolution(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          )
        )
        (norm2): MaisiGroupNorm3D(32, 64, eps=1e-06, affine=True)
        (conv2): MaisiConvolution(
          (conv): Convolution(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          )
        )
        (nin_shortcut): Identity()
      )
      (3): MaisiDownsample(
        (conv): MaisiConvolution(
          (conv): Convolution(
            (conv): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
          )
    

In [8]:
data_config = get_config((24, 128, 128)).data

datamodule = CTRATEDataModule(data_config)
dataloader = datamodule.val_dataloader()
len(dataloader)

valid:   0%|          | 0/500 [00:00<?, ?it/s]

No. of valid datapoints: 500


500

In [12]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        x = batch['image']
        x = x[:1]
        print(x.shape, x.min(), x.max())
        torch.cuda.reset_peak_memory_stats()
        # y = model.encode(x.to(device))[0].cpu()
        y = model(x.to(device))[0].cpu()
        mem = torch.cuda.max_memory_allocated()
        print(y.shape, y.min(), y.max())
        print(f"Memory: {mem / 2 ** 30:.2f} GB")

        for i in range(x.shape[0]):
            plot_scans([x[i, 0], y[i, 0]], ["Original", "Reconstructed"])
        
        break

torch.Size([1, 1, 24, 128, 128]) metatensor(0.0045) metatensor(1.)
torch.Size([1, 1, 24, 128, 128]) tensor(0.0011) tensor(1.1662)
Memory: 0.74 GB


interactive(children=(IntSlider(value=0, description='z', max=23), Output()), _dom_classes=('widget-interact',…

In [9]:
from monai.inferers.inferer import SlidingWindowInferer

inferer = SlidingWindowInferer((4, 64, 64), overlap=0.25)

In [10]:
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        x = batch['image']
        x = x[:1]
        print(x.shape, x.min(), x.max())
        torch.cuda.reset_peak_memory_stats()
        y = inferer(x.to(device), model)[0].cpu()
        mem = torch.cuda.max_memory_allocated()
        print(y.shape, y.min(), y.max())
        print(f"Memory: {mem / 2 ** 30:.2f} GB")

        for i in range(x.shape[0]):
            plot_scans([x[i, 0], y[i, 0]], ["Original", "Reconstructed"])
        
        break

torch.Size([1, 1, 24, 128, 128]) metatensor(0.) metatensor(0.6570)
torch.Size([1, 1, 24, 128, 128]) metatensor(-0.1219) metatensor(1.5047)
Memory: 0.11 GB


interactive(children=(IntSlider(value=0, description='z', max=23), Output()), _dom_classes=('widget-interact',…

In [21]:
n = 24
mip_x = x[:, :, :n].max(dim=2, keepdim=True).values
mip_y = y[:, :, :n].max(dim=2, keepdim=True).values

for i in range(mip_x.shape[0]):
    plot_scans([mip_x[i, 0], mip_y[i, 0]], [f"Original MIP {n} slices", f"Reconstructed MIP {n} slices"])

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…